<p style="font-family:Times New Roman;font-style: italic; font-weight: bold; font-size:300%; background-color:lightgrey; color:black; text-align:center;line-height : 40px; margin : 0; padding : 0;">
<br />
Migrate MySQL to MongoDB
</p>
<p style="font-family:Times New Roman;font-size:240%; background-color:lightgrey; color:black; text-align:center;line-height : 60px; margin : 0; padding : 0;">
Advanced Data Bases
</p>
<p style="font-family:Times New Roman; font-size:200%; text-align:center; line-height : 40px;  margin-top : 0; margin-bottom : 0; "> 
    <br /> By:
</p>

<p style="font-family:Times New Roman; font-size:120%; text-align:center; line-height: 15px;  margin-top : 0; margin-bottom : 0; "> 
    <b>DANIEL FELIPE MÉNDEZ PAEZ</b> <br />(<a href="mailto:dmendezpa@unal.edu.co">dmendezpa@unal.edu.co</a>)
</p>

<br />

<p style="font-family:Times New Roman; font-size:120%; text-align:center; line-height: 15px;  margin-top : 0; margin-bottom : 0; "> 
    <b>PAULO ANDRÉS GUEVARA POLANÍA</b> <br />(<a href="mailto:pguevarap@unal.edu.co">pguevarap@unal.edu.co</a>)
</p>



<p style="font-family:Times New Roman; font-size:180%; text-align:center; line-height : 30px;  margin-top : 0; margin-bottom : 0; "> 
    <br />
    <br />
    Engeneering Systems and Computing
</p>
<p style="font-family:Times New Roman; font-size:180%; text-align:center; line-height : 30px; margin-top : 0; ">
    Engenneering Faculty
</p>
<p style="font-family:Times New Roman; font-size:160%; text-align:center; line-height : 30px; margin-top : 0; ">
    Universidad Nacional de Colombia
</p>
<br>
<div align="center">
    <img src="images\logoUnall.png", width=300 height=300>
</div>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> 
    <br>
    Febrero de 2024
</p>

In [2]:
###############################################################
#!/usr/bin/python
# Purpose: Migrate Products from MySQL to Mongo
# Dependecies : python-pip , pymongo , mysqldb
# apt-get install python-pip ; pip install pymongo;
# apt-get install python-MySQLdb


import mysql.connector
import pymongo as mondb       # Import MongoDB Library
import decimal
import multiprocessing as mp  # Import Multiprocessing
from datetime import datetime


In [3]:
###############################################################
mysqldb = mysql.connector.connect(
    host="localhost",
    database="project_star_model",
    user="root",
    password="Danipau2124"
)
 
mycursor = mysqldb.cursor(dictionary=True)
mycursor.execute("SELECT * from warnings1")
myresult = mycursor.fetchall()
 
####print(myresult)

ProgrammingError: 1146 (42S02): Table 'project_star_model.warnings1' doesn't exist

In [1]:
###############################################################
import pymongo
##
mongodb_host = "mongodb://localhost:27017/"
mongodb_dbname = "mymongodb"
myclient = pymongo.MongoClient(mongodb_host)
mydb = myclient[mongodb_dbname]
mycol = mydb["eh"]
##
if len(myresult) > 0:
    x = mycol.insert_many(myresult) #myresult comes from mysql cursor
    print(len(x.inserted_ids))


NameError: name 'myresult' is not defined

-----

In [168]:
###############################################################
import mysql.connector
import pymongo
import datetime
import enum

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class MsgType(enum.Enum):
    HEADER = 1
    OKBLUE = 2
    OKCYAN = 3
    OKGREEN = 4
    WARNING = 5
    FAIL = 6
    ENDC = 7
    BOLD = 8
    UNDERLINE = 9

# Función de impresión bonita
def prettyprint(msg_text, msg_type):
    if msg_type == MsgType.HEADER:
        print(f"{bcolors.HEADER}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.OKBLUE:
        print(f"{bcolors.OKBLUE}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.OKCYAN:
        print(f"{bcolors.OKCYAN}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.OKGREEN:
        print(f"{bcolors.OKGREEN}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.WARNING:
        print(f"{bcolors.WARNING}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.FAIL:
        print(f"{bcolors.FAIL}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.BOLD:
        print(f"{bcolors.BOLD}{msg_text}{bcolors.ENDC}")
    elif msg_type == MsgType.UNDERLINE:
        print(f"{bcolors.UNDERLINE}{msg_text}{bcolors.ENDC}")

def convert_decimal_to_float(document):
    for key, value in document.items():
        if isinstance(value, decimal.Decimal):
            document[key] = float(value)
    return document

#Función migrate_table 
def migrate_table(db, table_name):
     # TODO: Arreglar el nombre de la tabla para cumplir con las restricciones de nomenclatura de la colección MongoDB
     # Por ejemplo, el signo $ está permitido en los nombres de las tablas MySQL pero no en los nombres de las colecciones MongoDB.
    
    mycursor = db.cursor(dictionary=True)
    mycursor.execute("SELECT * FROM " + table_name + ";")
    myresult = mycursor.fetchall()

    mycol = mydb[table_name]
    
    if delete_existing_documents:
        # eliminar todos los documentos de la colección
        mycol.delete_many({})

    for row in myresult:
        for key, value in row.items():
            if isinstance(value, datetime.date):
                row[key] = datetime.datetime(value.year, value.month, value.day)
            if isinstance(value, decimal.Decimal):
                row[key] = float(value)
                
    # insertar los documentos
    if len(myresult) > 0:
        x = mycol.insert_many(myresult)
        return len(x.inserted_ids)
    else:
        return 0

##Funcion migrate_index
## Esta funcion busca en las tablas dimensiones todo lo relacionado a un index
def migrate_index(db, table_name, FK, index_s,table_mongo):
 
    mycursor = db.cursor(dictionary=True)
    mycursor.execute("SELECT * FROM " + table_name + " WHERE " + FK + " = " + index_s +";")
    myresult = mycursor.fetchall()

    coleccion_original = mydb['table_mongo']

# Colección con los nuevos IDs
    coleccion_nuevos_ids = myresult[0]
    nuevo_id = None
    # Obtener el nuevo ID correspondiente al documento actual
    if isinstance(myresult[0], dict):
        nuevo_id = next((registro for registro in [myresult[0]] if registro.get(FK) == int(index_s)), None)
# Iterar sobre los documentos de la colección original
    coleccion_original.update_one({FK: int(index_s)}, {'$set': {FK: nuevo_id}})
            
    return nuevo_id


    """mycol = mydb[table_name]
    
    if delete_existing_documents:
        # eliminar todos los documentos de la colección
        mycol.delete_many({})

    # insertar los documentos
    if len(myresult) > 0:
        x = mycol.insert_many(myresult)
        return len(x.inserted_ids)
    else:
        return 0"""



In [62]:
begin_time = datetime.datetime.now()
abort = False
prettyprint(f"Script started at: {begin_time}", MsgType.HEADER)

################3
delete_existing_documents = False;
mysql_host="localhost"
mysql_database="project_star_model"
mysql_schema = "project_star_model"
mysql_user="root"
mysql_password="Danipau2124"


##
mongodb_host = "mongodb://localhost:27017/"
mongodb_dbname = "fhiudshfihdsf"
###################

if (delete_existing_documents):
    confirm_delete = input("Eliminar documentos existentes de las colecciones (S)Si/(N)No/(A)Abortar?")### Delete existing documents from collections (y)es/(n)o/(a)bort 
    if confirm_delete.lower() == "A":
        abort = True
    elif confirm_delete.lower() == "N":
        delete_existing_documents = False
    else:
        #Confirm again
        confirm_delete = input("Estas seguro (S) Si / (N) No?") ###  Are you sure (y)es/(n)
        if confirm_delete.lower() == "S":
            delete_existing_documents = True
        else:
            abort = True

if abort:
    prettyprint("Script abortado por el usuario", MsgType.FAIL)
else:
    if (delete_existing_documents):
        prettyprint("Los documentos existentes se eliminarán de las colecciones.", MsgType.FAIL)
    else:
        prettyprint("Los documentos existentes no se eliminarán de las colecciones.", MsgType.OKGREEN)
        
    #MySQL connection
    prettyprint("Conexión al servidor MySQL...", MsgType.HEADER)
    mysqldb = mysql.connector.connect(
        host=mysql_host,
        database=mysql_database,
        user=mysql_user,
        password=mysql_password
    )
    prettyprint("La conexión al servidor MySQL fue exitosa.", MsgType.OKGREEN)

    #MongoDB connection
    prettyprint("Conexión al servidor MongoDB...", MsgType.HEADER)
    myclient = pymongo.MongoClient(mongodb_host)
    mydb = myclient[mongodb_dbname]
    prettyprint("La conexión al servidor MongoDB fue exitosa.", MsgType.OKGREEN)

    # Arranca la migración
    prettyprint("La migración comenzó...", MsgType.HEADER)

    dblist = myclient.list_database_names()
    if mongodb_dbname in dblist:
        prettyprint("La base de datos existe.", MsgType.OKBLUE)
    else:
        prettyprint("La base de datos no existe, se está creando..", MsgType.WARNING)

    # Recorrer la lista de tablas en el esquema
    table_list_cursor = mysqldb.cursor()
    table_list_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s ORDER BY table_name LIMIT 15;", (mysql_schema,))
    #table_list_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s ORDER BY table_name LIMIT 15;")
    tables = table_list_cursor.fetchall()

    total_count = len(tables)
    success_count = 0
    fail_count = 0


    ## Se filtran las que se tienen que pasar enteras
    
    graduaciones = list(filter(lambda x: x[0] == 'graduaciones', tables))
    contrataciones = list(filter(lambda x: x[0] == 'contrataciones', tables))
    registro_publicaciones = list(filter(lambda x: x[0] == 'registro_publicaciones', tables))

    migrate_table(mysqldb, graduaciones[0][0])
    migrate_table(mysqldb, contrataciones[0][0])
    migrate_table(mysqldb, registro_publicaciones[0][0])

    ## Estas tablas tienen que estar embebidas dentro de las que pasaron enteras

    for table in tables:
        try:
            prettyprint(f"Procesando la tabla: {table[0]}...", MsgType.OKCYAN)
            inserted_count = migrate_table(mysqldb, table[0])
            print(table[0])
            success_count += 1
            prettyprint(f"Procesando la tabla: {table[0]} lista! Completada!. {inserted_count} documentos insertados.", MsgType.OKGREEN)
        except Exception as e:
            fail_count += 1
            prettyprint(f"{e}", MsgType.FAIL)

    prettyprint("Migration completed.", MsgType.HEADER)
    prettyprint(f"{success_count} de {total_count} tablas migradas satisfactoriamente.", MsgType.OKGREEN)
    if fail_count > 0:
        prettyprint(f"Migración de {fail_count} tablas no se pudieron realizar. Verifica los errores que surgieron a continuación.", MsgType.FAIL)
    
end_time = datetime.datetime.now()
prettyprint(f"Script completado a las : {end_time}", MsgType.HEADER)
prettyprint(f"Total tiempo de ejecución total fue de: {end_time-begin_time}", MsgType.HEADER)



Script started at: 2024-03-25 15:49:39.763773
Los documentos existentes no se eliminarán de las colecciones.
Conexión al servidor MySQL...
La conexión al servidor MySQL fue exitosa.
Conexión al servidor MongoDB...
La conexión al servidor MongoDB fue exitosa.
La migración comenzó...
La base de datos existe.
Procesando la tabla: cargos...
cargos
Procesando la tabla: cargos lista! Completada!. 2000 documentos insertados.
Procesando la tabla: contrataciones...
contrataciones
Procesando la tabla: contrataciones lista! Completada!. 100 documentos insertados.
Procesando la tabla: departamento...
departamento
Procesando la tabla: departamento lista! Completada!. 20 documentos insertados.
Procesando la tabla: egresado...
egresado
Procesando la tabla: egresado lista! Completada!. 100 documentos insertados.
Procesando la tabla: empresa...
empresa
Procesando la tabla: empresa lista! Completada!. 200 documentos insertados.
Procesando la tabla: facultad...
facultad
Procesando la tabla: facultad list

In [169]:
cargos = list(filter(lambda x: x[0] == 'cargos', tables))
ver1=migrate_index(mysqldb, cargos[0][0],'cargo_id', "338",'contrataciones')
mydb['contrataciones'].update_one({'cargo_id': 338}, {'$set': {'cargo_id': ver1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [26]:
resultado = list(filter(lambda x: x[0] == 'graduaciones', tables))
resultado

[('graduaciones',)]

In [27]:
print(resultado[0][0])

graduaciones


In [28]:
print(tables)

[('cargos',), ('contrataciones',), ('departamento',), ('egresado',), ('empresa',), ('facultad',), ('graduaciones',), ('institucion',), ('programa',), ('publicacion',), ('registro_publicaciones',), ('revista',), ('sede_universidad',), ('time_',)]


In [170]:
for documento in mydb['contrataciones'].find():
    # Imprimir cada documento
    print(documento)

{'_id': ObjectId('6600e32110f7d6242a11b3c6'), 'cargo_nombre': 'INVESTMENT BANKER, OPERATIONAL', 'cargo_id': 211}
{'_id': ObjectId('6600e32110f7d6242a11b3c7'), 'cargo_id': 211, 'empresa_id': 49, 'egresado_id': 15, 'date_id': datetime.datetime(1986, 8, 2, 0, 0), 'salario': 53605969.0, 'contratacion_id': 2}
{'_id': ObjectId('6600e32110f7d6242a11b3c8'), 'cargo_nombre': 'PRODUCT/PROCESS DEVELOPMENT SCIENTIST', 'cargo_id': 1654}
{'_id': ObjectId('6600e32110f7d6242a11b3c9'), 'cargo_id': {'cargo_nombre': 'DESIGNER, BLOWN GLASS/STAINED GLASS', 'cargo_id': 150}, 'empresa_id': 73, 'egresado_id': 19, 'date_id': datetime.datetime(2009, 2, 4, 0, 0), 'salario': 33574922.0, 'contratacion_id': 4}
{'_id': ObjectId('6600e32110f7d6242a11b3ca'), 'cargo_id': {'cargo_nombre': 'EDUCATIONAL PSYCHOLOGIST', 'cargo_id': 835}, 'empresa_id': 96, 'egresado_id': 87, 'date_id': datetime.datetime(2009, 12, 18, 0, 0), 'salario': 20071440.0, 'contratacion_id': 5}
{'_id': ObjectId('6600e32110f7d6242a11b3cb'), 'cargo_id': 

In [29]:
# MySQL Config
myConfig = {'host': '<host>',
            'username': '<user>',
            'password': '<passwd>',
            'db': '<db>'}


moncon = mondb.MongoClient('mongodb://<user>:<passwd>@<host>:27017/<db>')
mondb = moncon.db.collection

# Number of threads is 3 times the cpu on the machine i.e. 3 threads per cpu
threads = 3*mp.cpu_count()


# Insert Products to MongoDB
def index(product):
    productExist = mondb.find_one({'_id': product[0]})

    if productExist is None:
            mycon = mydb.connect(myConfig['host'], myConfig['username'],
                                 myConfig['password'], myConfig['db'])
            iProduct = {'_id': int(product[0]),
                        'sku': product[3],
                        'name': product[4],
                        'long_desc': product[2],
                        'short_desc': product[1],
                        'brand': {'id': int(product[5]), 'name': product[6]},
                        'style': {'id': int(product[10]), 'name': product[11]},
                        'category': {'id': int(product[7]), 'name': product[8],
                                     'parent': int(product[9])},
                        'in_stock': 1,
                        }

            iProduct['images'] = productImages(product[0], mycon)

            attributes = productAttributes(product[0], mycon)

            for attribute in attributes.iterkeys():
                iProduct[attribute] = attributes[attribute]

            prices = productPrices(product[0], mycon)
            iProduct['currency'] = int(prices['currency'])
            iProduct['prices'] = prices['prices']

            iProduct['sale_price'] = prices['prices']['sale']

            mondb.insert(iProduct)

            print 'Product: ' + str(product[0])+' indexed'

    else:
        exit()
        print 'Product: ' + str(product[0])+' already exists'

    return True


# Fetch Products from MySQL
def fetchproducts():
    mycon = mydb.connect(myConfig['host'], myConfig['username'],
                         myConfig['password'], myConfig['db'])
    with mycon:

        cur = mycon.cursor()
        cur.execute("SELECT p.id, p.short_desc, p.long_desc, p.sku, p.name, \
                    p.brand_id, b.brand_name, p.category_id, \
                    c.category_name, c.parent_category, \
                    b.style_id, b.style_name \
                    FROM products as p \
                    LEFT JOIN brands as b ON p.brand_id = b.id \
                    LEFT JOIN product_categories as c \
                    ON c.id = p.category_id ")
        products = cur.fetchall()
        return products


# Fetch Product Images
def productImages(id, mycon):
    with mycon:

        cur = mycon.cursor()
        cur.execute("SELECT i.path, i.type, i.label FROM product_images as i \
                    WHERE i.product_id =" + str(id))
        images = cur.fetchall()
        r = []
        for image in images:

            r.append({'path': image[0], 'type': image[1], 'label': image[2]})
        return r


# Fetch Product Attributes
def productAttributes(id, mycon):
    with mycon:

        cur = mycon.cursor()
        cur.execute("SELECT a.attribute_key, av.value, av.id \
                    FROM product_attribute_values as av \
                    LEFT JOIN  product_attributes as a \
                    ON a.id = av.attribute_id \
                    WHERE av.product_id =" + str(id))
        attributes = cur.fetchall()
        r = {}
        for attribute in attributes:
            if str(attribute[0]) == 'size':
                if 'size' not in r.keys():
                    r['size'] = []
                temp = rectify(id, attribute[2], attribute[1], mycon)

                if temp is not None:

                    if isinstance(temp, list):
                        for te in temp:
                            if te not in r['size']:
                                r['size'].append(te)
                    elif isinstance(temp, str):
                        if temp not in r['size']:
                            r['size'].append(temp)

            else:
                r[attribute[0]] = attribute[1]
        return r


# Fetch Product Prices
def productPrices(id, mycon):
    with mycon:

        cur = mycon.cursor()
        cur.execute("SELECT p.type, p.value, p.currency_id \
                    FROM product_prices as p \
                    WHERE p.product_id =" + str(id))
        prices = cur.fetchall()
        r = {'currency': 0, 'prices': {}}
        i = 0
        for price in prices:
            r['currency'] = price[2]
            r['prices'][price[0]] = str(price[1])
            i += 1
        return r


# Rectify/Purify Product Attribute 'Size'
def rectify(pid, av_id, value, mycon):
    r = value.split(',')
    if len(r) > 1:
        with mycon:
            cur = mycon.cursor()
            cur.execute('DELETE FROM product_attribute_values \
                        WHERE id = ' + str(av_id))
        temp = []
        for size in r:
            if '\xa3' not in size:
                temp.append(size)
                sql = 'INSERT INTO product_attribute_values \
                        (product_id, attribute_id, value) \
                        values (' + str(pid) + ", 9, '" + str(size) + "' )"
                cur.execute(sql)

        return temp

    elif '\xa3' in value:
        with mycon:
            cur = mycon.cursor()
            cur.execute('DELETE FROM product_attribute_values WHERE id = '
                        + str(av_id))
        return None
    else:
        return value


# Main Function
def main():
    products = fetchproducts()

    pool = mp.Pool(threads)
    for product in products:
        pool.apply_async(index, args=(product,))
    pool.close()
    pool.join()

# If you do not want MultiThreading uncomment the following line and comment
# the call of main function
# products = fetchproducts()

# for product in products:
#     index(product)

main()


SyntaxError: invalid syntax (866689332.py, line 49)